# 📊 AtliQ Hardware - Financial Performance Analysis Report

<h2 style="background-color:#2ecc71; color:white; padding:10px; border-radius:6px; text-align:center">
Table of Contents
</h2>

<a id='back'></a>

* [Introduction](#intro)   
*Executive Summary and Project Overview*
* [Stage 1. Data overview and preprocessing](#data_review)
    * [Conclusions](#data_review_conclusions)
    
* [Stage 2. Financial Performance Trends Analysis](#fin)
    
    * [2.1 Time-Based Metrics](#time)
        * [2.1.1 Revenue](#rev)
        * [2.1.2 Gross Profit](#gp)
        * [2.1.3 Gross Margin](#gm)
        * [2.1.4 CAGR](#cagr)
        * [Conclusions](#time_conclusions)
        
    * [2.2 Category-Based Metrics](#Category)
        * [2.2.1 Revenue](#cat_rev)
        * [2.2.2 Gross Profit](#cat_gp)
        * [2.2.3 Gross Margin](#cat_gm)
        * [Conclusions](#cat_conclusions)
        
    * [2.3 Markets](#market)
        * [2.3.1 Revenue](#market_rev)
        * [2.3.2 Gross Profit](#market_gp)
        * [2.3.3 Gross Margin](#market_gm)
        * [2.3.4 Market-level profitability matrix](#market_matrix)
        * [2.3.5 Annual revenue by market](#annual_rev)
        * [Conclusions](#market_conclusions)
        
    * [2.4 Waterfall Analysis](#wf)
    
    * [2.5 All Final Visualizations and conclusions](#fin_conclusions)
    
* [Stage 3. Formulating Hypothesis, A/B Test Analysis](#hypo)
    * [3.1 Formulating Hypothesis](#prior)
    * [3.2 A/B Test Analysis](#ab)
    * [3.3 Conclusions](#hypo_conclusions)
    
* [Stage 4. Financial Health Dashboard](#dash)
    * [4.1 The Dashboard](#dash)
    * [4.2 Conclusions](#dash_conclusions)

## Introduction <a id='intro'></a>
In this project we are working as junior analysts for the TTWC consulting agency ("TripleTen World Consultancy"). This year, we're dealing with a particularly special client: AtliQ Hardware.
AtliQ Hardware is one of the leading computer hardware producers in India, and has even expanded into other countries as well. They sell different types of hardware to big players such as Amazon, Best Buy, and Walmart. This year, they're asking PWC to conduct a big audit of their sales and help them automate their existing data. AtliQ Hardware has provided us with a backup of their database in the SQLite format. 

The clients are particularly interested in three business areas: Financial Analysis, Product Analysis and Customer Analysis. This EDA is focused on the first area - Financial Analysis.
The goal of the project is to analyze financial performance trends, identify market dynamics, and determine the most profitable segments in order to optimize revenue and profitability strategies.

The project will consist of four stages:
 1. Data overview and preprocessing 
 2. Financial Performance Trends Analysis
 3. Prioritizing Hypotheses and A/B Test Analysis
 4. Financial Health Dashboard prepared in Tableau


## Stage 1. Data overview and preprocessing <a id='data_review'></a>
Importing all the necessary libraries and reading the dataset from a csv file using pandas.
Obtaining the information about the table using different ways to check for missing values, duplicates and the correctness of column namings. 


In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from datetime import datetime
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import LinearSegmentedColormap
import plotly.graph_objects as go
import numpy as np
from IPython.display import Image, display
import os
from scipy import stats

In [ ]:
con = sqlite3.connect('atliq_db.sqlite3') 
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


In [ ]:
def download_check(table):
    con = sqlite3.connect("atliq_db.sqlite3")
    query = f"SELECT * FROM {table}"
    df = pd.read_sql_query(query, con)
    con.close()
    print(df.head(3))
    print ('-'*30)
    print(df.info())
    print("-" * 30)
    print(df.nunique())
    print("-" * 30)
    print(df.isna().sum())
    print("-" * 30)
    print('Duplicated:', df.duplicated().sum())
    print("-" * 30)
    print(df.columns)
    print("-" * 30)
    print(df.describe())
    return df

In [ ]:
dim_customer = download_check("dim_customer")

In [ ]:
dim_customer.groupby('market').nunique()

In [ ]:
dim_customer.groupby("customer").nunique().sort_values(by='platform', ascending=False).head(3)

In [ ]:
dim_customer['platform'] = dim_customer['platform'].astype("category")
dim_customer["channel"] = dim_customer["channel"].astype("category")

In [ ]:
dim_customer.info()

In [ ]:
dim_product = download_check("dim_product")

In [ ]:
dim_product['division'] = dim_product['division'].astype("category")
dim_product["segment"] = dim_product["segment"].astype("category")

In [ ]:
dim_product.info()

In [ ]:
fact_pre_discount = download_check("fact_pre_discount")

In [ ]:
fact_manufacturing_cost = download_check("fact_manufacturing_cost")

In [ ]:
fact_manufacturing_cost['product_code'].duplicated().sum()

In [ ]:
fact_gross_price = download_check("fact_gross_price")

In [ ]:
fact_gross_price['product_code'].duplicated().sum()

In [ ]:
fact_sales_monthly = download_check("fact_sales_monthly")

In [ ]:
fact_sales_monthly = fact_sales_monthly.dropna()

In [ ]:
fact_sales_monthly.isna().sum()

In [ ]:
fact_sales_monthly["date"] = pd.to_datetime(
    fact_sales_monthly["date"], format="%Y-%m-%d"
)
fact_sales_monthly["customer_code"] = fact_sales_monthly["customer_code"].astype(
    "int64"
)
fact_sales_monthly["sold_quantity"] = fact_sales_monthly["sold_quantity"].astype(
    "int64"
)
fact_sales_monthly["fiscal_year"] = fact_sales_monthly["fiscal_year"].astype("int64")

In [ ]:
fact_sales_monthly.info()

In [ ]:
sorted(fact_sales_monthly[fact_sales_monthly["fiscal_year"] == 2018]["date"].unique())

In [ ]:
sorted(fact_sales_monthly[fact_sales_monthly["fiscal_year"] == 2019]["date"].unique())

In [ ]:
sorted(fact_sales_monthly[fact_sales_monthly["fiscal_year"] == 2020]["date"].unique())

In [ ]:
sorted(fact_sales_monthly[fact_sales_monthly["fiscal_year"] == 2021]["date"].unique())

In [ ]:
sorted(fact_sales_monthly[fact_sales_monthly["fiscal_year"] == 2022]["date"].unique())

In [ ]:
fact_sales_monthly[fact_sales_monthly["fiscal_year"] == 2022].info()

In [ ]:
fact_sales_monthly_18_21 = fact_sales_monthly[~(fact_sales_monthly["fiscal_year"] == 2022)]
fact_sales_monthly_18_21.info()

#### Data Merging

In [ ]:
df = fact_sales_monthly_18_21.merge(dim_product, how='left')
df = df.drop(["division", "segment", "product", "variant"], axis=1)
df.head()

In [ ]:
df = df.merge(fact_manufacturing_cost, left_on=['product_code', 'fiscal_year'], right_on=['product_code', 'cost_year'], how="left")
df = df.drop(columns='cost_year')

In [ ]:
df = df.merge(fact_gross_price, how='left', on=['fiscal_year', 'product_code'])
df.head()

In [ ]:
df = df.merge(fact_pre_discount, how='left', on=['fiscal_year', 'customer_code'])
df.head()

In [ ]:
df = df.merge(dim_customer, how='left')
df.head()

In [ ]:
df_cust = df # the variable may be useful for deeper customer analysis

In [ ]:
df = df.drop(["customer", "platform", "channel", "sub_zone"], axis=1)
df.head()

## Conclusions <a id='data_review_conclusions'></a>
There are 6 tables in the dataset.  Below is a data dictionary that lists the columns in each table and describes the data that they hold. 

**1. dim_customer** - contains customer-related data
- 'customer_code': 	unique identifier of customer
- 'customer': company name of the customer
- 'platform': platform, through which the sale’s been done
- 'channel': channel of sale	
- 'market': country of the customer’s office
- 'sub_zone': abbreviation of the region
- 'region': region of the customer’s office

**2. dim_product** - contains product-related data
- 'product_code': unique identifier of product
- 'division': group of the product
- 'segment': type of product (sub-division)
- 'category': category of the product
- 'product': full product name
- 'variant': variant of the product

**3. fact_pre_discount** - contains pre-invoice deductions information for each product
- 'customer_code': unique identifier of customer
- 'fiscal_year': year when the discount was valid
- 'pre_invoice_discount_pct': discount % per invoice for specific customer

**4. fact_manufacturing_cost** - contains the cost incurred in the production of each product
- 'product_code': unique identifier of product
- 'cost_year': year of production
- 'manufacturing_cost': cost of production of unit of product

**5. fact_gross_price** - contains gross price information for each product
- 'product_code': unique identifier of product
- 'fiscal_year': year of transaction
- 'gross_price': final price for the product

**6. fact_sales_monthly** - contains monthly sales data for each product
- 'date': date of transaction
- 'product_code': unique identifier of product
- 'customer_code': unique identifier of customer
- 'sold_quantity': sold items to customer on that date
- 'fiscal_year': year of transaction

In our exploratory data analysis preprocessing phase, we successfully prepared the dataset for further analysis. The data integrity checks revealed some duplicate records and missing values, so that in order to perform a meaningfull analysis we had to get rid of 10% of the original data. We improved data organization by standardizing column names using a consistent naming convention. Additionally, we optimized data types for each variable to ensure appropriate storage and computational efficiency. These preprocessing steps have established a solid foundation for our subsequent analysis, ensuring that our insights will be based on properly structured and validated data.

Finally, we merged all the necessary for our future analysis data into a clean and complete dataset to work with, having the following structure:

**df** - contains the sales information, combined with markets, customers and products to reveal the whole picture 
- 'date': date of transaction              
- 'product_code': unique identifier of product             
- 'customer_code': unique identifier of customer        
- 'sold_quantity ': sold items to customer on that date           
- 'fiscal_year': year of transaction                
- 'category': category of the product               
- 'manufacturing_cost':  cost of production of unit of product       
- 'gross_price': final price for the product           
- 'pre_invoice_discount_pct': discount % per invoice for specific customer 
- 'market': country of the customer’s office    
- 'region': region of the customer’s office   

[Back to Contents](#back)

<h2 style="background-color:#2ecc71; color:white; padding:10px; border-radius:6px; text-align:center">
2. Business Metrics 📊
</h2>
<a id='fin'></a>

<h3 style="color:#588ac0; margin-top:20px">2.1 🕒 Time-Based Metrics </h3>
<a id='time'></a>

<h4 style="color:#588ac0">2.1.1 📈 Revenue </h4> 
<a id='rev'></a>

In [ ]:
df['revenue'] = df['sold_quantity']*df['gross_price']*(1-df['pre_invoice_discount_pct'])
df.head()

In [ ]:
df["revenue"] = (
    df['revenue'].round(4)
)

In [ ]:
revenue_y = df.groupby('fiscal_year')['revenue'].sum().round().reset_index()

In [ ]:
revenue_y

In [ ]:
rev = sns.barplot(x="fiscal_year", y="revenue", data=revenue_y, color="#2ecc71")
plt.title("Total Revenue by Fiscal Year", fontweight="semibold")
rev.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{int(x/1e6)}M"))
rev.set_xlabel("Fiscal Year")
rev.set_ylabel("Revenue (in millions)")
for index, row in revenue_y.iterrows():
    rev.text(index, row.revenue + 80000, f"{int(row.revenue)}$", ha="center")
plt.tight_layout()
plt.savefig("1.png", dpi=300, bbox_inches="tight")
plt.show()

AtliQ Hardware shows strong year-over-year revenue growth.

The revenue nearly doubled every year between 2018 and 2021. This upward trend indicates market expansion.

The consistent growth suggests a healthy and scalable business model.

In [ ]:
df["calendar_month"] = df["date"].dt.month
df["calendar_year"] = df["date"].dt.year
df["month_name"] = df["date"].dt.strftime("%B")

In [ ]:
def get_fiscal_month(date, fiscal_year):
    if date.month >= 9:  
        return date.month - 8
    else:  
        return date.month + 4

df["fiscal_month"] = df.apply(
    lambda row: get_fiscal_month(row["date"], row["fiscal_year"]), axis=1
)

fiscal_month_names = {
    1: "Sep",
    2: "Oct",
    3: "Nov",
    4: "Dec",
    5: "Jan",
    6: "Feb",
    7: "Mar",
    8: "Apr",
    9: "May",
    10: "Jun",
    11: "Jul",
    12: "Aug",
}
df["fiscal_month_name"] = df["fiscal_month"].map(
    fiscal_month_names
)

revenue_m = (
    df.groupby(["fiscal_year", "fiscal_month", "fiscal_month_name"])
    .agg(
        {
            "revenue": "sum", 
            "sold_quantity": "sum",
        }
    )
    .reset_index()
)

revenue_m = revenue_m.sort_values(["fiscal_year", "fiscal_month"])
revenue_m.head()


In [ ]:
revenue_m["revenue_change_pct"] = (
    revenue_m.groupby("fiscal_year")["revenue"].pct_change() * 100
)
revenue_m["revenue_change_abs"] = revenue_m.groupby("fiscal_year")["revenue"].diff()
revenue_m["cumulative_revenue"] = revenue_m.groupby("fiscal_year")["revenue"].cumsum()
revenue_m.head()

In [ ]:
sns.set_style("whitegrid")
color_map = {
    2018: "#ff4040",
    2019: "#ff8c00",
    2020: "#f1c40f",
    2021: "#2ecc71",
}
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("Monthly Revenue Analysis by Fiscal Year", fontsize=16, fontweight="bold")
ax1 = axes[0, 0]
for fy in revenue_m["fiscal_year"].unique():
    data = revenue_m[revenue_m["fiscal_year"] == fy]
    ax1.plot(
        data["fiscal_month"],
        data["revenue"],
        marker="o",
        linewidth=2,
        markersize=6,
        label=f"{int(fy)}",
        color=color_map[int(fy)]
    )

ax1.set_title("Monthly Revenue by Fiscal Year", fontweight="bold")
ax1.set_xlabel("Fiscal Month")
ax1.set_ylabel("Revenue")
ax1.set_xticks(range(1, 13))
ax1.set_xticklabels([fiscal_month_names[i] for i in range(1, 13)], rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)


ax2 = axes[0, 1]
pivot_changes = revenue_m.pivot(
    index="fiscal_month_name", columns="fiscal_year", values="revenue_change_pct"
)
fiscal_order = [
    "Sep",
    "Oct",
    "Nov",
    "Dec",
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
]
pivot_changes = pivot_changes.reindex(fiscal_order)

sns.heatmap(
    pivot_changes,
    annot=True,
    fmt=".1f",
    cmap="RdYlGn",
    center=0,
    ax=ax2,
    cbar_kws={"label": "Change %"},
)
ax2.set_title("Month-over-Month Revenue Changes (%)", fontweight="bold")
ax2.set_xlabel("Fiscal Year")
ax2.set_ylabel("Fiscal Month")

ax3 = axes[1, 0]
for fy in revenue_m["fiscal_year"].unique():
    data = revenue_m[revenue_m["fiscal_year"] == fy]
    ax3.plot(
        data["fiscal_month"],
        data["cumulative_revenue"],
        marker="s",
        linewidth=2,
        markersize=6,
        label=f"{int(fy)}",
        color=color_map[int(fy)]
    )

ax3.set_title("Cumulative Revenue by Fiscal Year", fontweight="bold")
ax3.set_xlabel("Fiscal Month")
ax3.set_ylabel("Cumulative Revenue")
ax3.set_xticks(range(1, 13))
ax3.set_xticklabels([fiscal_month_names[i] for i in range(1, 13)], rotation=45)
ax3.legend()
ax3.grid(True, alpha=0.3)

ax4 = axes[1, 1]
pivot_revenue = revenue_m.pivot(
    index="fiscal_month_name",
    columns="fiscal_year",
    values="revenue",
    )
pivot_revenue = pivot_revenue.reindex(fiscal_order)
colors = [color_map[year] for year in pivot_revenue.columns]
pivot_revenue.plot(kind="bar", ax=ax4, width=0.8, color=colors)
ax4.set_title("Revenue Comparison by Fiscal Month", fontweight="bold")
ax4.set_xlabel("Fiscal Month")
ax4.set_ylabel("Revenue")
ax4.legend(title="Fiscal Year")
ax4.tick_params(axis="x", rotation=45)

def millions(x, pos):
    return f"{x*1e-6:.1f}M"


formatter = FuncFormatter(millions)
for ax in [ax1, ax3, ax4]:
    ax.yaxis.set_major_formatter(formatter)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.savefig("2.png", dpi=300, bbox_inches="tight")
plt.show()

**Key insights**

* Revenue in 2021 consistently outperforms previous years across all months.
* From January onward, 2021 maintains a consistently high monthly revenue base (~1.3M), reflecting operational maturity or a sustained growth initiative.
* Fiscal years 2020-2021 showed a steady increase, particularly from October to December. There is a strong seasonal spike, indicating likely holiday or promotional impacts.
* Earlier years (2018–2019) have much lower cumulative totals, proving consistent year-over-year growth of the business we saw on the previous graph. 2018 shows the lowest and flattest revenue trend, suggesting either early-stage operations or low growth. 
* October and November appear to be consistently strong revenue months across all fiscal years.
* December is consistently the highest revenue month, especially for 2020 and 2021.
* January-March period exhibits relatively lower revenue, suggesting seasonal market slowdown.
* April-June shows revenue recovery, indicating cyclical business patterns.
* Stabilization trend appears towards the end of each fiscal year (Jun-Aug).

[Back to Contents](#back)

In [ ]:
df.info()

<h4 style="color:#588ac0">2.1.2 💰 Gross Profit </h4>  
<a id='gp'></a>

In [ ]:
df.head()

In [ ]:
df['gp'] = df['revenue']-df['manufacturing_cost']*df['sold_quantity']
df.head(1)

In [ ]:
df["gp"] = (
    df['gp'].round(4)
)

In [ ]:
gp_y = df.groupby('fiscal_year')['gp'].sum().round().reset_index()

In [ ]:
gp_y

In [ ]:
gp = sns.barplot(x="fiscal_year", y="gp", data=gp_y, color="#2ecc71")
plt.title("Gross Profit by Fiscal Year", fontweight="semibold")
gp.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{int(x/1e6)}M"))
gp.set_xlabel("Fiscal Year")
gp.set_ylabel("Gross Profit (in millions)")
for index, row in gp_y.iterrows():
    gp.text(index, row.gp + 80000, f"{int(row.gp)}$", ha="center")
plt.tight_layout()
plt.savefig("3.png", dpi=300, bbox_inches="tight")
plt.show()

Gross profit follows the revenue growth trend closely.
Profitability scales with volume, which supports AtliQ’s current pricing and cost strategy.


In [ ]:
gp_m = (
    df.groupby(["fiscal_year", "fiscal_month", "fiscal_month_name"])
    .agg(
        {
            "gp": "sum"
        }
    )
    .reset_index()
)

gp_m = gp_m.sort_values(["fiscal_year", "fiscal_month"])
gp_m.head()


In [ ]:
gp_m["gp_change_pct"] = (
    gp_m.groupby("fiscal_year")["gp"].pct_change() * 100
)
gp_m["gp_change_abs"] = gp_m.groupby("fiscal_year")["gp"].diff()
gp_m["cumulative_gp"] = gp_m.groupby("fiscal_year")["gp"].cumsum()
gp_m.head()

In [ ]:
sns.set_style("whitegrid")
color_map = {
    2018: "#ff4040",
    2019: "#ff8c00",
    2020: "#f1c40f",
    2021: "#2ecc71",
}
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle(
    "Monthly Gross Profit Analysis by Fiscal Year", fontsize=16, fontweight="bold"
)
ax1 = axes[0, 0]
for fy in gp_m["fiscal_year"].unique():
    data = gp_m[gp_m["fiscal_year"] == fy]
    ax1.plot(
        data["fiscal_month"],
        data["gp"],
        marker="o",
        linewidth=2,
        markersize=6,
        label=f"{int(fy)}",
        color=color_map[int(fy)],
    )

ax1.set_title("Monthly Gross Profit by Fiscal Year", fontweight="bold")
ax1.set_xlabel("Fiscal Month")
ax1.set_ylabel("Gross Profit")
ax1.set_xticks(range(1, 13))
ax1.set_xticklabels([fiscal_month_names[i] for i in range(1, 13)], rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[0, 1]
pivot_changes = gp_m.pivot(
    index="fiscal_month_name", columns="fiscal_year", values="gp_change_pct"
)
fiscal_order = [
    "Sep",
    "Oct",
    "Nov",
    "Dec",
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
]
pivot_changes = pivot_changes.reindex(fiscal_order)

sns.heatmap(
    pivot_changes,
    annot=True,
    fmt=".1f",
    cmap="RdYlGn",
    center=0,
    ax=ax2,
    cbar_kws={"label": "Change %"},
)
ax2.set_title("Month-over-Month Gross Profit Changes (%)", fontweight="bold")
ax2.set_xlabel("Fiscal Year")
ax2.set_ylabel("Fiscal Month")

ax3 = axes[1, 0]
for fy in gp_m["fiscal_year"].unique():
    data = gp_m[gp_m["fiscal_year"] == fy]
    ax3.plot(
        data["fiscal_month"],
        data["cumulative_gp"],
        marker="s",
        linewidth=2,
        markersize=6,
        label=f"{int(fy)}",
        color=color_map[int(fy)],
    )

ax3.set_title("Cumulative Gross Profit by Fiscal Year", fontweight="bold")
ax3.set_xlabel("Fiscal Month")
ax3.set_ylabel("Cumulative Gross Profit")
ax3.set_xticks(range(1, 13))
ax3.set_xticklabels([fiscal_month_names[i] for i in range(1, 13)], rotation=45)
ax3.legend()
ax3.grid(True, alpha=0.3)

ax4 = axes[1, 1]
pivot_gp = gp_m.pivot(
    index="fiscal_month_name",
    columns="fiscal_year",
    values="gp",
)
pivot_gp = pivot_gp.reindex(fiscal_order)
colors = [color_map[year] for year in pivot_gp.columns]
pivot_gp.plot(kind="bar", ax=ax4, width=0.8, color=colors)
ax4.set_title("Gross Profit Comparison by Fiscal Month", fontweight="bold")
ax4.set_xlabel("Fiscal Month")
ax4.set_ylabel("Gross Profit")
ax4.legend(title="Fiscal Year")
ax4.tick_params(axis="x", rotation=45)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.savefig("4.png", dpi=300, bbox_inches="tight")
plt.show()

**Key insights**
* Profit tracks revenue but with sharper dips during weak months.
* Seasonality is again evident, especially in in October-November across most years, likely aligned with festival seasons and hardware refresh cycles. January shows consistent decline, suggesting post-holiday market correction.
* The heatmap reveals substantial month-over-month improvements. The April growth of 238.7% is due to the exceptional profit decline in the previous month. 
* 2021 maintains steady performance levels without the extreme volatility seen in 2020, indicating business maturation and more predictable revenue streams.
* Insights could guide improved planning of promotions or pricing.

<h4 style="color:#588ac0">2.1.3 📉 Gross Margin </h4> 
<a id='gm'></a>

In [ ]:
margin_y = df.groupby('fiscal_year')[['gp', 'revenue']].sum().reset_index()
margin_y

In [ ]:
margin_y["margin"] = (margin_y['gp']/margin_y['revenue']*100).round(3)
margin_y

In [ ]:
mar = sns.barplot(x="fiscal_year", y="margin", data=margin_y, color="#2ecc71")
plt.title("Gross Margin by Fiscal Year", fontweight="semibold")
mar.set_xlabel("Fiscal Year")
mar.set_ylabel("Gross Margin")
mar.set_ylim(0, 100)
for index, row in margin_y.iterrows():
    mar.text(index, row.margin +0.7, f"{(row.margin)}%", ha="center")
plt.tight_layout()
plt.savefig("5.png", dpi=300, bbox_inches="tight")
plt.show()

Despite rapid growth, gross margins remain remarkably stable around 61%, ranging from 61.031% to 61.607%. 

This consistency demonstrates strong pricing discipline and cost management throughout the expansion period.

In [ ]:
margin_m = gp_m.merge(
    revenue_m, on=["fiscal_year", "fiscal_month", "fiscal_month_name"]
)[
    [
        "fiscal_year",
        "fiscal_month",
        "fiscal_month_name",
        "gp",
        "revenue",
        "sold_quantity",
    ]
].reset_index()

margin_m = margin_m.sort_values(["fiscal_year", "fiscal_month"])
margin_m.head()

In [ ]:
margin_m["margin"] = (
    margin_m["gp"] / margin_m["revenue"] * 100
).round(3)
margin_m.head()

In [ ]:
margin_m["margin_change_pct"] = (
    margin_m.groupby("fiscal_year")["margin"].pct_change() * 100
)
margin_m.head()

In [ ]:
sns.set_style("whitegrid")

color_map = {
    2018: "#ff4040",
    2019: "#ff8c00",
    2020: "#f1c40f",
    2021: "#2ecc71",
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle(
    "Monthly Gross Margin and Sold Quantity Analysis by Fiscal Year",
    fontsize=16,
    fontweight="bold",
    y=1.05
)

for fy in margin_m["fiscal_year"].unique():
    data = margin_m[margin_m["fiscal_year"] == fy]
    ax1.plot(
        data["fiscal_month"],
        data["margin"],
        marker="o",
        linewidth=2,
        markersize=6,
        label=f"{int(fy)}",
        color=color_map[int(fy)],
    )

ax1.set_title("Monthly Gross Margin by Fiscal Year", fontweight="bold")
ax1.set_xlabel("Fiscal Month")
ax1.set_ylabel("Gross Margin (%)")
ax1.set_xticks(range(1, 13))
ax1.set_xticklabels([fiscal_month_names[i] for i in range(1, 13)], rotation=45)
ax1.legend(title="Fiscal Year")
ax1.grid(True, alpha=0.3)

pivot_changes = margin_m.pivot(
    index="fiscal_month_name", columns="fiscal_year", values="margin_change_pct"
)

fiscal_order = [
    "Sep", "Oct", "Nov", "Dec", "Jan", "Feb",
    "Mar", "Apr", "May", "Jun", "Jul", "Aug"
]
pivot_changes = pivot_changes.reindex(fiscal_order)

sns.heatmap(
    pivot_changes,
    annot=True,
    fmt=".1f",
    cmap="RdYlGn",
    center=0,
    ax=ax2,
    cbar_kws={"label": "Change (%)"},
    linewidths=0.3,
    linecolor="gray",
    annot_kws={"size": 9}
)
ax2.set_title("MoM Gross Margin Change (%)", fontweight="bold")
ax2.set_xlabel("Fiscal Year")
ax2.set_ylabel("Fiscal Month")

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig("6.png", dpi=300, bbox_inches="tight")
plt.show()

**Key insights**
* Margins are relatively stable across months, except for slight dips in high-sales periods. This indicates possible discounting or volume-driven promotions during peaks.
* The February peaks in 2018-2019 possibly indicate a decision to rise profits during typically slower sales periods. This could have driven to lower revenue, so in later years we don't see that high volatility.
* 2020-2021 show reduced margin volatility compared to previous years, indicating improved operational efficiency and pricing strategies. The margin range narrowed from 1.1% (2018-2019) to approximately 0.4% (2020-2021).
* Overall, margin management across months appears effective. No critical volatility — positive sign for cost control.


<h4 style="color:#588ac0; margin-top:20px">2.1.4 📈 CAGR – Compound Annual Growth Rate </h3>
<a id='cagr'></a>


In [ ]:
def calculate_cagr(start, end, periods):
    return ((end / start) ** (1 / periods)) - 1

start_value = df[df["fiscal_year"] == 2018]["revenue"].sum()
end_value = df[df["fiscal_year"] == 2021]["revenue"].sum()
cagr = calculate_cagr(start_value, end_value, periods=3)
print(f"CAGR: {cagr:.2%}")

AtliQ Hardware achieved an extraordinary compound annual growth rate of 103.32% from 2018-2021, effectively doubling revenue each year during this period. This exceptional growth rate indicates the company successfully scaled from startup phase to established market player, likely driven by strong demand in the computer hardware sector and effective partnerships with major retail channels.

<h4 style="color:#588ac0">📝 Conclusions</h4> 
<a id='time_conclusions'></a>
AtliQ Hardware exhibits remarkable financial performance with consistent high-growth trajectory while maintaining stable profitability margins. The business shows strong seasonal patterns that can be leveraged for inventory planning and marketing strategies. The company has successfully scaled operations without sacrificing profit margins, indicating effective cost management and pricing strategies. The maturation from high volatility growth in 2020 to more stable, predictable performance in 2021 suggests the company is transitioning from startup-phase rapid expansion to sustainable business operations.

<h3 style="color:#eb6123; margin-top:30px;">2.2 🏷️ Category-Based Metrics</h3>
<a id='Category'></a>

In [ ]:
cat = df.groupby('category')[['gp', 'revenue']].sum().reset_index()
cat

<h4 style="color:#eb6123"> 2.2.1 📈 Revenue </h4> 
<a id='cat_rev'></a>

In [ ]:
rev_cat = sns.barplot(x="category", y="revenue", data=cat, color="#2ecc71")
plt.title("Total Revenue by Category", fontweight="semibold")
rev_cat.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{int(x/1e6)}M"))
rev_cat.set_xlabel("Category")
rev_cat.set_ylabel("Revenue (in millions)")
for index, row in cat.iterrows():
    rev_cat.text(index, row.revenue + 300000, f"{int(row.revenue)}$", ha="center")
plt.tight_layout()
plt.savefig("7.png", dpi=300, bbox_inches="tight")
plt.show()

Internal HDD represents AtliQ's primary revenue driver, generating $30.59M compared to Graphic Cards at $5.75M. 

This creates an 84:16 revenue split, indicating heavy dependence on storage solutions business.

<h4 style="color:#eb6123"> 2.2.2💰 Gross Profit </h4> 
<a id='cat_gp'></a>

In [ ]:
gp_cat = sns.barplot(x="category", y="gp", data=cat, color="#2ecc71")
plt.title("Total Gross Profit by Category", fontweight="semibold")
gp_cat.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{int(x/1e6)}M"))
gp_cat.set_xlabel("Category")
gp_cat.set_ylabel("Gross Profit (in millions)")
for index, row in cat.iterrows():
    gp_cat.text(index, row.gp + 300000, f"{int(row.gp)}$", ha="center")
plt.tight_layout()
plt.savefig("8.png", dpi=300, bbox_inches="tight")
plt.show()

Internal HDD delivers $18.72M in gross profit versus $3.51M from Graphic Cards, maintaining the same proportional relationship as revenue. 

This consistency indicates similar operational efficiency across both product lines.

<h4 style="color:#eb6123">2.2.3 📉 Gross Margin</h4> 
<a id='cat_gm'></a>

In [ ]:
cat['margin'] = (cat['gp']/cat['revenue']*100).round(3)

In [ ]:
margin_cat = sns.barplot(x="category", y="margin", data=cat, color="#2ecc71")
plt.title("Total Margin Profit by Category", fontweight="semibold")
margin_cat.set_xlabel("Category")
margin_cat.set_ylabel("Gross Margin %")
margin_cat.set_ylim(0, 100)
for index, row in cat.iterrows():
    margin_cat.text(index, row.margin+1, f"{row.margin}%", ha="center")

plt.tight_layout()
plt.savefig("9.png", dpi=300, bbox_inches="tight")
plt.show()

 Both categories demonstrate nearly identical gross margins - Graphic Cards at 61.058% and Internal HDD at 61.179%.
 
 This 0.12% difference indicates consistent pricing strategies and cost structures across product lines.

<h4 style="color:#eb6123">2.2.4 📝 Conclusions</h4> 
<a id='cat_conclusions'></a>
AtliQ Hardware operates a highly concentrated business model with Internal HDD driving 84% of revenue and profit. While this concentration presents risks from market volatility or technological disruption in storage solutions, it also demonstrates AtliQ's competitive strength and market leadership in this segment. The identical gross margins across categories indicate strong operational discipline and suggests the company could potentially scale the Graphic Card business with similar profitability if market opportunities arise. The current portfolio structure provides a stable foundation while the smaller Graphic Card segment offers diversification potential for future growth.

<h3 style="color:#229756; margin-top:30px;"> 2.3 🌍 Markets</h3>
<a id='market'></a>

In [ ]:
market_metrics = df.groupby("market")[["gp", "revenue", 'sold_quantity']].sum().reset_index()
market_metrics

<h4 style="color:#229756"> 2.3.1 📈 Revenue </h4> 
<a id='market_rev'></a>

In [ ]:
market_rev_sorted = market_metrics.sort_values(by="revenue", ascending=False)
plt.figure(figsize=(12, 8))
sns.barplot(
    data=market_rev_sorted,
    x="revenue",
    y="market",
    color="#2ecc71",
)
plt.title("Revenue by Market (in millions)", fontsize=14, fontweight="bold")
plt.xlabel("Revenue")
plt.ylabel("Market")
for index, value in enumerate(market_rev_sorted["revenue"]):
    plt.text(value + 1e5, index, f"{value/1e6:.1f}M", va="center")
formatter = FuncFormatter(lambda x, _: f"{x/1e6:.0f}M")
plt.gca().xaxis.set_major_formatter(formatter)
plt.grid(axis="x", alpha=0.3)
plt.tight_layout()
plt.savefig("10.png", dpi=300, bbox_inches="tight")
plt.show()

* India dominates with 10.4M revenue - nearly 2x larger than USA
* USA holds second position at 6.0M
* South Korea ranks third at 3.4M
* eavy concentration: Top 3 markets generate ~55% of total revenue
* Long tail of smaller markets (many under 1M)

In [ ]:
market_metrics["market_share_pct"] = (
    market_metrics["revenue"] / market_metrics["revenue"].sum()
) * 100

market_sorted = market_metrics.sort_values("market_share_pct", ascending=True)

fig, ax = plt.subplots(figsize=(10, 8)) 
bars = ax.barh(
    range(len(market_sorted)),
    market_sorted["market_share_pct"],
    color="#2ecc71",
    linewidth=0.5,
)

for i, (idx, row) in enumerate(market_sorted.iterrows()):
    ax.text(
        row["market_share_pct"] + 0.5,
        i,
        f"{row['market_share_pct']:.1f}%",
        va="center",
        fontsize=9,
    )

ax.set_yticks(range(len(market_sorted)))
ax.set_yticklabels(market_sorted["market"], fontsize=10)
ax.set_xlabel("Market Share %", fontsize=12)
ax.set_title("Market Share by Revenue", fontsize=14, fontweight="bold")
ax.grid(alpha=0.3, axis="x")
ax.set_xlim(0, market_sorted["market_share_pct"].max() * 1.15)

plt.tight_layout()
plt.savefig("12.png", dpi=300, bbox_inches="tight")
plt.show()

* India commands 28.7% market share
* USA holds 16.5%
* Top 3 markets (India, USA, South Korea) control 54.6% of total market
* High fragmentation: 20+ markets with <2% share each

<h4 style="color:#229756"> 2.3.2💰 Gross Profit </h4> 
<a id='market_gp'></a>

In [ ]:
market_gp_sorted = market_metrics.sort_values(by="gp", ascending=False)
plt.figure(figsize=(10, 8))
sns.barplot(
    data=market_gp_sorted,
    x="gp",
    y="market",
    color="#2ecc71",
)
plt.title("Gross Profit by Market (in millions)", fontsize=14, fontweight="bold")
plt.xlabel("Gross Profit")
plt.ylabel("Market")
for index, value in enumerate(market_gp_sorted["gp"]):
    plt.text(value + 30000, index, f"{value/1e6:.1f}M", va="center")
formatter = FuncFormatter(lambda x, _: f"{x/1e6:.0f}M")
plt.gca().xaxis.set_major_formatter(formatter)
plt.grid(axis="x", alpha=0.3)
plt.tight_layout()
plt.savefig("13.png", dpi=300, bbox_inches="tight")
plt.show()

* India leads gross profit at 6.4M
* USA second at 3.7M
* South Korea third at 2.1M
* Profit distribution mirrors revenue structure, confirming margin consistency
* Clear tier structure: premium markets vs. developing markets

<h4 style="color:#229756">2.3.3 📉 Gross Margin</h4> 
<a id='market_gm'></a>

In [ ]:
market_metrics["margin"] = (
    market_metrics["gp"] / market_metrics["revenue"] * 100
).round(3)

market_metrics.head()

In [ ]:
market_mar_sorted = market_metrics.sort_values(by="margin", ascending=False)
plt.figure(figsize=(10, 8))
sns.barplot(
    x="margin", y="market", data=market_mar_sorted, color="#2ecc71"
)
plt.title("Gross Margin by Market", fontweight="semibold")
plt.xlabel("Market")
plt.ylabel("Gross Margin")
plt.xlim(0, 100)

plt.tight_layout()
plt.savefig("14.png", dpi=300, bbox_inches="tight")
plt.show()

* Gross margins are remarkably consistent across markets (~60-65%)
* Indonesia leads with the highest gross margin
* Mexico shows the lowest margin among displayed markets
* Narrow margin spread of only 5-7 percentage points between best and worst performers
* Indicates strong pricing discipline and cost management across regions

<h4 style="color:#229756">2.3.4 📈 Market-level profitability matrix</h4> 
<a id='market_matrix'></a>

In [ ]:
market_metrics["avg_gross_margin"] = (market_metrics["gp"] / market_metrics["revenue"]) * 100

median_volume = market_metrics["sold_quantity"].median()
median_margin = market_metrics["avg_gross_margin"].median()

def categorize_market(row):
    volume = row["sold_quantity"]
    margin = row["avg_gross_margin"]

    if volume >= median_volume and margin >= median_margin:
        return "High Margin, High Volume"
    elif volume >= median_volume and margin < median_margin:
        return "Low Margin, High Volume"
    elif volume < median_volume and margin >= median_margin:
        return "High Margin, Low Volume"
    else:
        return "Low Margin, Low Volume"

market_metrics["quadrant"] = market_metrics.apply(categorize_market, axis=1)

colors = {
    "Low Margin, High Volume": "gold",
    "High Margin, High Volume": "green",
    "High Margin, Low Volume": "blue",
    "Low Margin, Low Volume": "red",
}

markers = {
    "Low Margin, High Volume": "o",
    "High Margin, High Volume": "X",
    "High Margin, Low Volume": "s",
    "Low Margin, Low Volume": "P",
}

plt.figure(figsize=(14, 10))

for category in colors.keys():
    data = market_metrics[market_metrics["quadrant"] == category]
    plt.scatter(
        data["sold_quantity"],
        data["avg_gross_margin"],
        c=colors[category],
        marker=markers[category],
        s=100,
        label=category,
        alpha=0.7,
        edgecolors="black",
        linewidth=0.5,
    )

    for idx, row in data.iterrows():
        plt.annotate(
            row["market"],
            (row["sold_quantity"], row["avg_gross_margin"]),
            xytext=(5, 5),
            textcoords="offset points",
            fontsize=9,
            alpha=0.8,
        )

plt.axvline(x=median_volume, color="gray", linestyle="--", alpha=0.7)
plt.axhline(y=median_margin, color="gray", linestyle="--", alpha=0.7)

plt.xlabel("Total Volume Sold", fontsize=12)
plt.ylabel("Average Gross Margin", fontsize=12)
plt.title("Market-Level Profitability Matrix", fontsize=16, fontweight="bold")

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")

plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f"{y:.1f}%"))

plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:,.0f}"))

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("15.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n" + "=" * 50)
print("Top-5 Revenue Markets:")
print(
    market_metrics.nlargest(5, "sold_quantity")[
        ["market", "sold_quantity", "avg_gross_margin"]
    ]
)

print("\nTop-5 Gross Margin Markets:")
print(
    market_metrics.nlargest(5, "avg_gross_margin")[
        ["market", "avg_gross_margin", "sold_quantity"]
    ]
)

*This analysis reveals a well-structured but India-dependent portfolio with clear strategic priorities for each market segment.*

* **High Margin, High Volume (Green X) - Premium Markets:**

Indonesia, Australia, Norway, France, Italy, Germany, Philippines, South Korea, USA
Best performers: Ideal combination of efficiency and scale


* **Low Margin, High Volume (Yellow O) - Volume Markets:**

India, Canada, United Kingdom
India is critical: Massive volume compensates for lower margins


* **High Margin, Low Volume (Blue Square) - Niche Markets:**

Bangladesh, Chile, New Zealand, Sweden, Pakistan
Development opportunity: Potential to scale up volume


* **Low Margin, Low Volume (Red Cross) - Challenge Markets:**

Mexico, Austria, Japan, Spain, Portugal, Colombia, Netherlands, Brazil
Requires attention: Either improve margins or increase volume


**Portfolio Optimization:**

* Protect premium markets (Green X): Maintain competitive advantage
* Scale volume markets (Yellow O): Focus on operational efficiency, especially India
* Develop niche markets (Blue Square): Investment in market development
* Fix or exit challenge markets (Red Cross): Strategic review needed

**Operational Priorities:**

* India strategy: Despite lower margins, massive volume makes it irreplaceable
* Premium market defense: Protect high-margin, high-volume markets from competition
* Margin improvement: Focus on red cross markets - either improve efficiency or consider market exit
* Volume expansion: Blue square markets show margin potential - invest in growth

**Risk Assessment:**

* Heavy dependence on India creates portfolio risk despite profitability
* Premium markets cluster suggests competitive positioning strength
* Challenge markets may be dragging overall performance

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
sizes = market_metrics["market_share_pct"] * 20
colors = market_metrics["sold_quantity"]

scatter = ax.scatter(
    market_metrics["avg_gross_margin"],
    market_metrics["gp"],
    s=sizes,
    c=colors,
    cmap="RdYlGn", 
    alpha=0.7,
    edgecolors="black",
    linewidth=0.5,
)

for idx, row in market_metrics.iterrows():
    ax.annotate(
        row["market"],
        (row["avg_gross_margin"], row["gp"]),
        xytext=(13, 0),
        textcoords="offset points",
        fontsize=9,
        alpha=0.8,
    )

ax.set_xlabel("Margin %", fontsize=12)
ax.set_ylabel("Profit", fontsize=12)
ax.set_title(
    "Profit vs Margin (size = market share, color = quantity)",
    fontsize=14,
    fontweight="bold",
)
ax.grid(alpha=0.3)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f"{y:,.0f}"))
cbar1 = plt.colorbar(scatter, ax=ax)
cbar1.set_label("Total Quantity", rotation=270, labelpad=20)

plt.tight_layout()
plt.savefig("16.png", dpi=300, bbox_inches="tight")
plt.show()

* India: Massive outlier with highest profit (6.4M), moderate margin (~61%), largest market share, and highest quantity (darkest green)
* USA: Strong profit (3.7M) with similar margin to India, large bubble indicating significant market share
* South Korea: Good profit (2.1M) with moderate margin (~61%), medium-sized market presence

<h4 style="color:#229756">2.3.5 Annual revenue by market</h4> 
<a id='annual_rev'></a>

In [ ]:
market_annual = df.groupby(["market",'fiscal_year'])[["gp", "revenue"]].sum().reset_index()
market_annual.head()

In [ ]:
market_annual['revenue'].describe()

In [ ]:
pivot = market_annual.pivot_table(
        values="revenue", index="market", columns="fiscal_year", fill_value=0
    )

pivot = pivot.loc[pivot.sum(axis=1).sort_values(ascending=True).index]
colors = [color_map.get(year, "#cccccc") for year in pivot.columns]
ax = pivot.plot(kind="barh", figsize=(16, 14), width=0.8, color=colors, alpha=0.8)

plt.title("Revenue by Market and Year", fontsize=14, fontweight="bold")
plt.xlabel("Revenue (millions)", fontsize=12, fontweight="bold")
plt.ylabel("Market", fontsize=12, fontweight="bold")
plt.legend(title="Fiscal Year", bbox_to_anchor=(1.05, 1))
plt.grid(axis="x", alpha=0.3)
plt.tight_layout()
plt.savefig("17.png", dpi=300, bbox_inches="tight")
plt.show()

The chart demonstrates strong portfolio diversification with clear market leaders driving overall growth while emerging markets contribute incremental expansion.
* **Market leaders:** India and USA dominate with revenues exceeding 3-5 million by 2021
* **Growth trajectories:** Most markets show consistent year-over-year expansion
* **Market maturity levels:** Clear distinction between established markets (USA, India, South Korea) and emerging ones (Colombia, Chile, Austria)
* **Revenue concentration:** Top 5 markets account for majority of total revenue

In [ ]:
pivot_rev = market_annual.pivot(index="market", columns="fiscal_year", values="revenue")

plt.figure(figsize=(10, 8))
sns.heatmap(
    pivot_rev, annot=True, fmt=".0f", cmap="YlGn", linewidths=0.3, linecolor="gray"
)
plt.title("Revenue by Market and Fiscal Year", fontsize=14, fontweight="bold")
plt.xlabel("Fiscal Year")
plt.ylabel("Market")
plt.tight_layout()
plt.savefig("18.png", dpi=300, bbox_inches="tight")
plt.show()

Clear visualization of market prioritization with established markets generating substantial revenues while newer markets are in development phases.
* **Revenue concentration:** India (5.1M) and USA (3.1M) represent the darkest cells, indicating highest absolute revenues in 2021
* **Growth patterns:** Progressive color intensification from 2018 to 2021 across most markets
* **Market gaps:** Several countries show no revenue in earlier years, indicating market entry timing
* **Scale differences:** Significant revenue disparities between mature and developing markets

In [ ]:
pivot_rev_safe = pivot_rev.replace(0, 1e-10)
pivot_change = pivot_rev_safe.pct_change(axis=1, fill_method=None) * 100
pivot_change = pivot_change.iloc[:, 1:]
pivot_change = pivot_change.clip(lower=-100, upper=1000)

mask = pivot_change.isna() | np.isinf(pivot_change)
plt.figure(figsize=(12, 8))
sns.heatmap(
    data=pivot_change,
    annot=True,
    fmt=".1f",
    cmap="RdYlGn",
    center=0,
    mask=mask,
    linewidths=0.3,
    linecolor="gray",
    cbar_kws={"label": "YoY Change (%)", "format": "%.0f%%"},
    vmin=-100,
    vmax=100
)

plt.title("YoY Revenue Change by Market (%)", fontsize=14, fontweight="bold")
plt.xlabel("Fiscal Year")
plt.ylabel("Market")
plt.tight_layout()
plt.savefig("19.png", dpi=300, bbox_inches="tight")
plt.show()

* India, France, and Canada show strong consistent YoY growth.
* Markets like Mexico, Pakistan, and Chile are declining or stagnant.

This suggests where to invest or reduce effort. Growth is concentrated — diversification remains a risk.

<h4 style="color:#229756">📝 Conclusions</h4> 
<a id='market_conclusions'></a>

**Strategic Priorities:**

* India - Core market requiring maximum investment and attention
* USA & South Korea - Critical secondary markets for stability
* Asia-Pacific dominance - Regional strength in key growth markets

**Operational Insights:**

* Consistent margins indicate effective global pricing strategy
* Revenue concentration creates dependency risk on top markets
* Opportunity exists in developing smaller markets with proven margin potential
* Geographic diversification provides risk mitigation

<h3 style="color:#8199cf; margin-top:20px">2.4 🌊 Waterfall Analysis </h3>
<a id='wf'></a>

In [ ]:
years = sorted(df["fiscal_year"].unique())

waterfalls = []

for i in range(len(years) - 1):
    y0 = years[i]
    y1 = years[i + 1]

    df0 = df[df["fiscal_year"] == y0]
    df1 = df[df["fiscal_year"] == y1]

    p0 = df0["gross_price"].mean()
    p1 = df1["gross_price"].mean()
    c0 = df0["manufacturing_cost"].mean()
    c1 = df1["manufacturing_cost"].mean()
    d0 = df0["pre_invoice_discount_pct"].fillna(0).mean()
    d1 = df1["pre_invoice_discount_pct"].fillna(0).mean()
    q0 = df0["sold_quantity"].sum()
    q1 = df1["sold_quantity"].sum()

    revenue0 = (1 - d0) * p0 * q0
    cost0 = c0 * q0
    gp0 = revenue0 - cost0

    revenue1 = (1 - d1) * p1 * q1
    cost1 = c1 * q1
    gp1 = revenue1 - cost1

    delta_gp = gp1 - gp0

    price_effect = (p1 - p0) * q0 * (1 - d0)
    cost_effect = (c0 - c1) * q0
    discount_effect = (d0 - d1) * p0 * q0  
    volume_effect = (q1 - q0) * ((1 - d0) * p0 - c0)
    residual = delta_gp - (price_effect + cost_effect + discount_effect + volume_effect)

    fig = go.Figure(
        go.Waterfall(
            name="Gross Profit YoY",
            orientation="v",
            measure=[
                "absolute",
                "relative",
                "relative",
                "relative",
                "relative",
                "total",
            ],
            x=[f"GP {y0}", "Price", "Cost", "Discount", "Volume", f"GP {y1}"],
            textposition="outside",
            y=[gp0, price_effect, cost_effect, discount_effect, volume_effect, gp1],
            connector={"line": {"color": "gray"}},
        )
    )

    fig.update_layout(
        title=f"💰 Gross Profit Waterfall: {y0} → {y1}",
        yaxis_title="Gross Profit",
        width=1200,
        height=300,
        showlegend=False,
    )

    waterfalls.append(fig)
    fig.write_image(f"20-{y0}_to_{y1}.png", width=1200, height=500, scale=2)
    fig.show()

**2018 → 2019**
* Gross profit rose primarily due to volume growth
* Price effect was slightly negative, suggesting pricing pressure
* Discounts had neutral effect — possible flat structure
* Cost improved marginally, supporting bottom-line growth

**2019 → 2020**
* Strong profit increase, with volume as the main driver again
* Price contributed positively — pricing strategy improved
* Cost and discount both slightly eroded profits

**2020 → 2021**
* Largest YoY GP growth
* Volume again is the top contributor
* Discount effect remains negative — may indicate aggressive promotions
* Cost control slipped slightly — opportunity to review procurement or operations


**Strategic Implications:** 

The waterfall analysis reveals a business in rapid scaling mode with strong fundamentals - consistent volume growth, pricing power, and managed cost scaling. The pattern suggests sustainable growth with opportunity for operational efficiency improvements.

## 2.5 📊 All Final Visualizations and conclusions <a id='fin_conclusions'></a>
[Download all graphs (PDF)](/graphs.pdf)

In [ ]:
image_folder = "graphs"

image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(".png")])

for file in image_files:
    print({file})
    display(Image(filename=os.path.join(image_folder, file), width=1200))


# 🎯 Final Strategic Conclusion

## Executive Summary

AtliQ Hardware has demonstrated exceptional business performance from 2018-2021, achieving a remarkable compound annual growth rate of 103.32% while maintaining stable gross margins around 61%. The company has successfully transitioned from a startup-phase operation to an established market player with predictable revenue streams and strong operational discipline.

## Key Performance Highlights

**Financial Excellence:**
- Revenue growth from $2.24M (2018) to $18.85M (2021) - an 8.4x increase
- Gross profit scaled from $1.36M to $11.51M with consistent 61% margins
- Evolved from high-volatility growth (2020) to stable, predictable performance (2021)
- Strong seasonal patterns with October-December representing peak performance periods

**Market Position:**
- Geographic concentration with India leading (28.7% market share, $10.4M revenue)
- Successful expansion across 20+ international markets
- Consistent gross margins (60-65%) across all geographic regions

## Strategic Portfolio Analysis

**Market Segmentation:**
- **Premium Markets (High Margin, High Volume):** Indonesia, Australia, Norway, France, Italy, Germany, Philippines, South Korea, USA - representing ideal performance quadrant
- **Volume Markets (Low Margin, High Volume):** India, Canada, United Kingdom - critical for scale despite margin pressure
- **Niche Markets (High Margin, Low Volume):** Bangladesh, Chile, New Zealand, Sweden, Pakistan - development opportunities
- **Challenge Markets (Low Margin, Low Volume):** Mexico, Austria, Japan, Spain, Portugal, Colombia, Netherlands, Brazil - requiring strategic review

**Product Concentration:**
- Both categories demonstrate consistent operational efficiency with nearly identical gross margins
- Heavy dependence on Internal HDD (84% revenue) creates both strength and risk
- Graphic Cards segment (16% revenue) offers diversification potential with identical 61% margins

## Growth Drivers & Operational Excellence

**Volume Leadership:** Consistently the primary driver of gross profit growth across all periods, indicating strong market demand and successful scaling strategies

**Pricing Power:** Positive price contribution in all growth periods demonstrates effective market positioning and competitive advantage

**Cost Management:** Proportional cost scaling relative to growth, though increasing pressures suggest need for operational efficiency focus

**Promotional Discipline:** Minimal discount impact indicates strategic promotional approach without margin erosion

## Strategic Recommendations

**Immediate Priorities:**
1. **Protect Core Markets:** Maintain competitive advantage in premium markets while defending India's massive volume contribution
2. **Diversify Risk:** Reduce dependence on Internal HDD through strategic expansion of Graphic Cards and new product categories
3. **Optimize Challenge Markets:** Implement strategic review for low-margin, low-volume markets - either improve efficiency or consider selective market exit
4. **Leverage Seasonality:** Capitalize on predictable Q2 strength (Oct-Dec) while developing initiatives to boost Q3-Q4 performance (Jan-Mar)

**Long-term Strategic Vision:**
- **Geographic Expansion:** Scale niche markets with proven margin potential while maintaining operational efficiency
- **Product Diversification:** Leverage identical margin structure to expand beyond storage solutions
- **Operational Maturity:** Continue transition from high-growth volatility to sustainable, predictable business operations
- **Market Leadership:** Consolidate position as regional hardware leader while expanding global footprint

## Risk Mitigation

**Concentration Risk:** Heavy dependence on India (28.7% market share) and Internal HDD (84% revenue) requires diversification strategy

**Market Volatility:** Seasonal fluctuations demand robust cash flow management and inventory planning

**Competitive Pressure:** Maintain pricing power through continued innovation and operational excellence

## Final Assessment

AtliQ Hardware represents a success story of rapid, sustainable growth with strong operational fundamentals. The company has achieved the challenging balance of high growth rates while maintaining profitability and operational discipline. The transition from startup volatility to mature, predictable performance positions AtliQ for continued success in the competitive hardware market.

The strategic foundation is solid: proven ability to scale operations, consistent margin management across diverse markets, and strong seasonal patterns that enable effective planning. With focused execution on diversification, operational efficiency, and strategic market development, AtliQ Hardware is well-positioned to maintain its growth trajectory while building long-term market leadership.

**Strategic Outlook:** Strong operational foundation with identified growth opportunities and recognized risk areas requiring strategic attention.

### 📎 Appendix: Full Graphs (PDF)



[Back to Contents](#back)


<h2 style="background-color:#6027ae; color:white; padding:10px; border-radius:6px; text-align:center">
3. Prioritizing Hypotheses, A/B Test Analysis
</h2>
<a id='hypo'></a>


<h3 style="color:#8199cf; margin-top:20px">3.1 Formulating Hypothesis </h3>
<a id='prior'></a>

**Hypothesis:** Margins remain the same during high periods (Oct-Dec) and declines (Jan-Mar)

H₀ (Null Hypothesis): Gross margins during high-revenue periods (Oct-Dec) are equal to gross margins during low-revenue periods (Jan-Mar)

H₁ (Alternative Hypothesis): Gross margins during high-revenue periods (Oct-Dec) are significantly different from gross margins during low-revenue periods (Jan-Mar)


<h3 style="color:#8199cf; margin-top:20px">3.2 A/B Testing </h3>
<a id='ab'></a>

In [ ]:
df['gross_margin_pct'] = (df['gp'] / df['revenue']) * 100

In [ ]:
peak_data = df[df["calendar_month"].isin([10, 11, 12])]  # Oct-Dec
decline_data = df[df["calendar_month"].isin([1, 2, 3])]  # Jan-Mar

In [ ]:
peak_monthly = (
    peak_data.groupby(["fiscal_year", "calendar_month"])
    .agg({"gp": "sum", "revenue": "sum"})
    .reset_index()
)

In [ ]:
decline_monthly = (
    decline_data.groupby(["fiscal_year", "calendar_month"])
    .agg({"gp": "sum", "revenue": "sum"})
    .reset_index()
)

In [ ]:
peak_monthly["margin"] = (peak_monthly["gp"] / peak_monthly["revenue"]) * 100
decline_monthly["margin"] = (decline_monthly["gp"] / decline_monthly["revenue"]) * 100

In [ ]:
peak_margins = peak_monthly["margin"].values
decline_margins = decline_monthly["margin"].values


In [ ]:
print("=== DESCRIPTIVE STATISTICS ===")
print(f"Peak months (Oct-Dec):")
print(f"  Mean: {np.mean(peak_margins):.2f}%")
print(f"  Std: {np.std(peak_margins):.2f}%")
print(f"  Count: {len(peak_margins)}")

print(f"\nDecline months (Jan-Mar):")
print(f"  Mean: {np.mean(decline_margins):.2f}%")
print(f"  Std: {np.std(decline_margins):.2f}%")
print(f"  Count: {len(decline_margins)}")

In [ ]:
t_stat, p_value = stats.ttest_ind(peak_margins, decline_margins)

In [ ]:
print(f"\n=== TEST RESULTS ===")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

In [ ]:
alpha = 0.05
if p_value < alpha:
    print(f"\n❌ HYPOTHESIS REJECTED (p < {alpha})")
    print("Peak and decline margins are SIGNIFICANTLY DIFFERENT")

    if np.mean(peak_margins) > np.mean(decline_margins):
        print("→ Peak margins are HIGHER than decline margins")
        print("→ You have pricing power during high demand periods")
    else:
        print("→ Peak margins are LOWER than decline margins")
        print("→ Possibly excessive discounting during peak periods")

else:
    print(f"\n✅ HYPOTHESIS SUPPORTED (p ≥ {alpha})")
    print("Peak and decline margins are NOT significantly different")
    print("→ Excellent pricing discipline!")
    print("→ Consistent operational efficiency")

In [ ]:
plt.figure(figsize=(10, 6))

# Box plots
plt.subplot(1, 2, 1)
plt.boxplot([peak_margins, decline_margins], tick_labels=["Peak", "Decline"])
plt.title("Margin Distribution")
plt.ylabel("Margin %")

# Time series
plt.subplot(1, 2, 2)
plt.plot(range(len(peak_margins)), peak_margins, "ro-", label="Peak")
plt.plot(range(len(decline_margins)), decline_margins, "bo-", label="Decline")
plt.title("Margins Over Time")
plt.ylabel("Margin %")
plt.legend()

plt.tight_layout()
plt.show()

print(f"\n📊 Monthly details:")
print("Peak months:")
print(peak_monthly[["fiscal_year", "calendar_month", "margin"]])
print("\nDecline months:")
print(decline_monthly[["fiscal_year", "calendar_month", "margin"]])

<h3 style="color:#8199cf; margin-top:20px">3.3 Conclusions </h3>
<a id='hypo_conclusions'></a>

The hypothesis test comparing gross margins during high-revenue months (October–December) and low-revenue months (January–March) yielded a p-value greater than 0.05, indicating no statistically significant difference between the two periods.

✅ This supports the null hypothesis, suggesting that the company maintains consistent margins across seasonal demand fluctuations.

📌 Key implications:

Excellent pricing discipline: The company successfully avoids margin erosion even when demand is high.

Operational efficiency: Cost structures and discounting strategies appear to be well-managed and stable throughout the fiscal year.

This consistency strengthens overall profitability and reduces risk related to seasonal volatility.

👉 No immediate intervention is needed in pricing or cost strategies across these periods, though continued monitoring is recommended to maintain this performance.

[Back to Contents](#back)


<h2 style="background-color:black; color:white; padding:10px; border-radius:6px; text-align:center">
4. Financial Health Dashboard
</h2>
<a id='dash'></a>

In [ ]:
df.to_csv('atliq.csv', index=False, encoding="utf-8")

<h3 style="color:#8199cf; margin-top:20px">4.1 The Dashboard</h3>
<a id='dash'></a>

### 📎 You can view the interactive Financial Health Dashboard [here](https://public.tableau.com/shared/TZM5WJZMY?:display_count=n&:origin=viz_share_link)

In [ ]:
image_folder = "dash screens"

image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(".png")])

for file in image_files:
    print({file})
    display(Image(filename=os.path.join(image_folder, file), width=1200))

**Dashboard Description:**

The dashboard provides a multi-dimensional view of AtliQ Hardware’s financial performance across markets, time periods, and product categories. The user can dynamically toggle between three key metrics — Revenue, Gross Profit and Gross Margin — and between two product categories — Graphic Cards and Internal HDDs, having an opportunity to choose the range of fiscal years.

It includes:

* Global market performance map: Highlights top-performing regions with exact revenue or profit figures.

* Revenue breakdown by market: Lists countries sorted by total revenue.

* Monthly and weekday trends: Shows how revenue and gross profit evolve across the calendar year and across the week.

* Time series graph: Compares revenue, gross profit and gross margin trends over time.

* Category distribution pie chart: Displays the contribution of each product line to total revenue.


<h3 style="color:#8199cf; margin-top:20px">4.2 Conclusions</h3>
<a id='dash_conclusion'></a>

**Key Insights and Conclusions:**

Seasonality and Demand Patterns:

* Revenue and gross profit exhibit strong seasonality, peaking in November and December across all years.

* A significant drop is observed in January through March, confirming a demand decline post-peak season.

* Weekday sales patterns differ throughout the years and categories. It is worth mentioning that in 2018 there were no sales on Saturdays. Apparently, the company made changes on the operating days to boost sales.

Consistent Margins Across Seasons:

* Despite fluctuations in sales volume, gross margins remain relatively stable throughout high and low seasons. This suggests excellent pricing discipline and strong cost control, reinforcing operational efficiency.

Market-Level Performance:

* India and the USA dominate in both revenue and gross profit.

* High variance exists across regions, suggesting potential to optimize marketing or distribution in underperforming markets like South America and parts of Europe.

Category Contribution:

* The pie chart shows that Internal HDDs contribute significantly more revenue and profit than Graphic Cards*, with potential implications for category-specific pricing or inventory strategies.



[Back to Contents](#back)